## Recurrent Neural Networks

Language is an inherently temporal phenomenon. Spoken language is a sequence of acoustic events over time, and we comprehend and produce both spoken and written language as a sequential input stream. The temporal nature of language is reflected in the metaphors we use; we talk of the flow of conversations, news feeds, and twitter streams, all of which emphasize that language is a sequence that unfolds in time. 

Yet most of the machine learning approaches we’ve studied so far, like those for sentiment analysis and other text classification tasks don’t have this temporal nature – they assume simultaneous access to all aspects of their input. The feedforward networks also assumed simultaneous access, although they also had a simple model for time. Recall that we applied feedforward networks to language modeling by having them look only at a fixed-size window of words, and then sliding this window over the input, making independent predictions along the way.

This section introduces a deep learning architecture that offers an alternative way of representing time: recurrent neural networks (RNNs), and their variants like LSTMs. RNNs have a mechanism that deals directly with the sequential nature of language, allowing them to handle the temporal nature of language without the use of arbitrary fixed-sized windows. The recurrent network offers a new way to represent the prior context, in its recurrent connections, allowing the model’s decision to depend on information from hundreds of words in the past.

A recurrent neural network (RNN) is any network that **contains a cycle** within its network connections, meaning that the value of some unit is directly, or indirectly, dependent on its own earlier outputs as an input. While powerful, such networks are difficult to reason about and to train. However, within the general class of recurrent networks there are constrained architectures that have proven to be extremely effective when applied to language. In this section, we consider a class of recurrent networks referred to as Elman Networks (Elman, 1990) or simple recurrent net works. These networks are useful in their own right and serve as the basis for more complex approaches like the Long Short-Term Memory (LSTM) networks. In this chapter when we use the term RNN we’ll be referring to these simpler more constrained networks (although you will often see the term RNN to mean any net with recurrent properties including LSTMs).

<div style="display:flex;align-items:center;justify-content:center;">
<img src="images/simple_rnn.png"  style="width:400px;object-fit:cover;"/>
</div>

As with ordinary feedforward networks, an input vector representing the current input, $x_t$, is multiplied by a weight matrix and then passed through a non-linear activation function to compute the values for a layer of hidden units. This hidden layer is then used to calculate a corresponding output, $y_t$. In a departure from our earlier window-based approach, sequences are processed by presenting one item at a time to the network. We’ll use subscripts to represent time, thus $x_t$ will mean the input vector $x$ at time $t$. The key difference from a feedforward network lies in the recurrent link shown in the figure with the dashed line. This link augments the input to the computation at the hidden layer with the value of the hidden layer from the preceding point in time.

The hidden layer from the previous time step provides a form of memory, or context, that encodes earlier processing and informs the decisions to be made at later points in time. Critically, this approach does not impose a fixed-length limit on this prior context; the context embodied in the previous hidden layer can include information extending back to the beginning of the sequence.

Adding this temporal dimension makes RNNs appear to be more complex than non-recurrent architectures. But in reality, they’re not all that different. Given an input vector and the values for the hidden layer from the previous time step, we’re still performing the standard feedforward calculation. To see this, consider below figure which clarifies the nature of the recurrence and how it factors into the computation at the hidden layer. The most significant change lies in the new set of weights, $U$, that connect the hidden layer from the previous time step to the current hidden layer. These weights determine how the network makes use of past context in calculating the output for the current input. As with the other weights in the network, these connections are trained via backpropagation.

<div style="display:flex;align-items:center;justify-content:center;">
<img src="images/rnn_architecture.png"  style="width:800px;object-fit:cover;"/>
</div>

### Inference in RNNs

Forward inference (mapping a sequence of inputs to a sequence of outputs) in an RNN is nearly identical to what we’ve already seen with feedforward networks. To compute an output $y_t$ for an input $x_t$, we need the activation value for the hidden layer $h_t$. To calculate this, we multiply the input $x_t$ with the weight matrix $W$, and the hidden layer from the previous time step $h_{t−1}$ with the weight matrix $$U$. We add these values together and pass them through a suitable activation function, $g$, to arrive at the activation value for the current hidden layer, $h_t$. Once we have the values for the hidden layer, we proceed with the usual computation to generate the output vector.

$$
h_t = g(Wh_{t-1} + Wx_t) \\
y_t = f(Vh_t)
$$

Let’s refer to the input, hidden and output layer dimensions as $d_\text{in}$, $d_\text{h}$, and $d_\text{out}$ respectively. Given this, our three parameter matrices are: $W \in \Reals^{d_\text{h} \times d_\text{in}}$, $U \in \Reals^{d_\text{h} \times d_\text{h}}$, and $V \in \Reals^{d_\text{out} \times d_\text{h}}$.

We compute $y_t$ via a softmax computation that gives a probability distribution over the possible output classes.
$$
y_t = \text{softmax}(Vh_t)
$$

The fact that the computation at time $t$ requires the value of the hidden layer from time $t−1$ mandates an incremental inference algorithm that proceeds from the start of the sequence to the end. The sequential nature of simple recurrent networks can also be seen by unrolling the network in time. In this figure, the various layers of units are copied for each time step to illustrate that they will have differing values over time. However, the various weight matrices are shared across time.

<div style="display:flex;align-items:center;justify-content:center;">
<img src="images/rnn_unrolled.png"  style="width:800px;object-fit:cover;"/>
</div>

<pre style="width:900px;border:1px solid black;">
<b>function</b> ForwardRNN(x, <i>network</i>) <b>returns</b> output sequence <b>y</b>
    <b>h</b><sub>0</sub> &larr; 0
    <b>for</b> i &larr; 1 <b>to</b> Length(<b>x</b>) <b>do</b>
        <b>h</b><sub>i</sub> &larr; g(<b>Uh</b><sub>i-1</sub> + <b>Wx</b><sub>i</sub>)
        <b>y</b><sub>i</sub> &larr; f(<b>Vh</b><sub>i</sub>)
    <b>return</b> y
<pre>

### Training

Just like feedforward networks, training recurrent neural networks (RNNs) involves using a training set, a loss function, and backpropagation to compute the gradients needed for weight updates. However, unlike feedforward networks, RNNs require updating three sets of weights:  
- **W**: Weights connecting the input layer to the hidden layer.  
- **U**: Weights connecting the previous hidden state to the current hidden state.  
- **V**: Weights connecting the hidden layer to the output layer.  

A key distinction in RNN training is that each hidden state depends on both the previous hidden state and the current input. This interdependence introduces challenges that do not exist in standard backpropagation. Specifically, the loss function for the output at time step **t** relies on the hidden state from time **t−1**, and the hidden state at **t** influences not only the output at **t** but also the hidden state at **t+1** (and subsequently the output at **t+1**). Consequently, computing the gradients for a given hidden state requires accounting for its impact on both the current and future outputs.  

To accommodate these dependencies, training RNNs follows a two-pass approach known as **backpropagation through time (BPTT)**:  
1. **Forward Pass**: The network processes the input sequence step by step, computing hidden states and outputs, accumulating the loss at each step, and storing hidden states for later use.  
2. **Backward Pass**: The sequence is processed in reverse, computing gradients for each time step while propagating errors backward through the sequence.  

Modern computational frameworks simplify this process by unrolling the recurrent network into an equivalent feedforward computational graph. This approach removes explicit recurrences, allowing standard backpropagation to be applied directly. By defining a template that includes the network's structure, weight matrices, and activation functions, it becomes possible to generate an unrolled computational graph specific to a given input sequence. This enables efficient forward inference and training using conventional optimization techniques.  

For applications involving long sequences—such as speech recognition, character-level language modeling, or streaming data—unrolling the entire sequence may be impractical due to computational constraints. Instead, the input can be divided into manageable fixed-length segments, treating each as a separate training instance. This allows RNNs to be trained effectively without excessive memory consumption.

### RNNs as Language Models

Let’s see how to apply RNNs to the language modeling task. Recall that language models predict the next word in a sequence given some preceding context. For example, if the preceding context is “Thanks for all the” and we want to know how likely the next word is “fish” we would compute:

$$
P(\text{fish}|\text{Thanks for all the})
$$

Language models give us the ability to assign such a conditional probability to every possible next word, giving us a distribution over the entire vocabulary. We can also assign probabilities to entire sequences by combining these conditional probabilities with the chain rule:

$$
P(w_{1:n}) = \prod_{i=1}^n P(w_i|w_{<i})
$$

The n-gram language models compute the probability of a word given counts of its occurrence with the $n−1$ prior words. The context is thus of size $n−1$.

For the feedforward language models, the context is the window size.

RNN language models (Mikolov et al., 2010) process the input sequence one word at a time, attempting to predict the next word from the current word and the previous hidden state. RNNs thus don’t have the limited context problem that n-gram models have, or the fixed context that feedforward language models have, since the hidden state can in principle represent information about all of the preceding words all the way back to the beginning of the sequence.